# Demo of a weighted generative model
An example weighted generative model implementation
To do:
1. Put the new optimiser into the genreative model // review current implementation
2. Implement Jax

In [ ]:
# weighted_genereative_model conda environment
# import requirements
import os
import numpy as np
import scipy
from scipy import optimize
import networkx as nx
import bct as bctpy
import matplotlib.pyplot as plt
import pylab
import copy
import random
from mpl_toolkits.mplot3d import Axes3D
import brainconn as bct

In [ ]:
# set network variables
nnode = 40 # number of nodes
xnode = 10 # x axis
ynode = 2 # y axis
znode = 2 # z axis
# compute inputs
Wtgt = np.tril(np.random.randint(0,100,(nnode,nnode)),-1)
Wtgt = (Wtgt+Wtgt.T)
threshold, upper, lower = 95,1,0
Atgt = np.where(Wtgt>threshold,upper,lower)
Aseed = np.zeros((nnode,nnode)) # seed the seed as a random edge that exists
#ind = np.argwhere(Atgt)
#indedge = ind[random.randrange(len(ind)),:]
#Aseed[indedge[0],indedge[1]] = Aseed[indedge[1],indedge[0]] = 1
x = np.arange(xnode) # set the x axis
y = np.arange(ynode) # set the y axis
z = np.arange(znode) # set the z axis
c = np.array(np.meshgrid(x,y,z)).T.reshape(-1,3) # placed in a grid
d = scipy.spatial.distance.pdist(c)
D = scipy.spatial.distance.squareform(d) # euclidean distance matrix
m = int(np.size(np.where(Atgt.flat))/2) # number of connections
eta = np.array([-2]) # eta parameter
gamma = np.array([.5]) # gamma parameter
model_type = 'matching' # generative model
model_var = 'powerlaw' # wiring equation
epsilon = 1e-6

In [ ]:
# visualise the seed matrix
subax1 = plt.subplot(121)
plt.imshow(Aseed)
plt.xlabel('Node')
plt.ylabel('Node')
# visualise the target matrix
subax1 = plt.subplot(122)
plt.imshow(Atgt)
plt.xlabel('Node')
plt.ylabel('Node')

In [ ]:
# run and example binary generative model
b = bct.generative.generative_model(Aseed, 
                     D, 
                     m, 
                     eta, 
                     gamma, 
                     model_type='matching',
                     model_var='powerlaw', 
                     epsilon=1e-6, 
                     copy=True)

In [ ]:
# visualise the binary model
# plot the outcome
plt.figure(figsize=(15,5))
subax1 = plt.subplot(131)
plt.imshow(Aseed)
subax2 = plt.subplot(132)
plt.imshow(Atgt)
subax2 = plt.subplot(133)
plt.imshow(b)
# display the number of edges in target and final network as a sanity check
m,np.size(np.where(b.flat))//2 # number of connections

In [ ]:
# define objective function of
def objective_function(x): # minimise communicability
    p = np.reshape(x,(nnode,nnode)) # reshape to a matrix form
    q = scipy.linalg.expm(p) # compute the communicability
    r = np.sum(q) # commute the total communicability
    return 2*r # << review!
# add in constraints
def constraint1_function(x): # while maintaining strengths 
    s = np.sum(x)
    return -s

In [ ]:
# let's use the old weighted matrix and use this to optimise
Wexamp = Wtgt/100
plt.imshow(Wexamp)
plt.colorbar()
print(objective_function(Wexamp.flatten()))

In [ ]:
# form the bounds
lubnds = (epsilon,1.0) # sets lower and upper bounds
bnds = ((lubnds,)*np.power(nnode,2)) # for each weight
# from the constraints
con1 = {'type':'ineq','fun':constraint1_function} # set constraints
cons = [con1] # defines them here

In [ ]:
# run the optimizer
solution = optimize.minimize(objective_function,b.flatten(),method='SLSQP',\
                             bounds=bnds,constraints=cons)
# NOTE - what we are running the optimizer on!
# print solution
print(solution)

In [ ]:
# plot the nework and its optimisation
plt.figure(figsize=(15,5))
y = np.reshape(solution.x,(nnode,nnode))
subax1 = plt.subplot(121)
plt.imshow(b)
plt.title('Simulation')
subax2 = plt.subplot(122)
plt.imshow(y)
plt.title('Optimisation')
plt.colorbar()
# note that the optimisation plateuas at the number of nodes! 
# This may give reason to think why we grow networks at all in the first place

In [ ]:
# initialise the weighted generative model
alpha = 100 # set the update rate
mi, _, _ = bct.similarity.matching_ind(Aseed) # matching index of the seed
Kseed = mi + mi.T # value matrix of the seed
Kseed += epsilon # add epsilon
mseed = np.size(np.where(Aseed.flat))//2 # compute the number of edges in the seed

In [ ]:
# run an example generative model
if type(model_var) == tuple:
    mv1, mv2 = model_var
else:
    mv1, mv2 = model_var, model_var

if mv1 in ('powerlaw', 'power_law'):
    Fd = D**eta
elif mv1 in ('exponential',):
    Fd = np.exp(eta*D)

if mv2 in ('powerlaw', 'power_law'):
    Fk = Kseed**gamma
elif mv2 in ('exponential',):
    Fk = np.exp(gamma*Kseed)
Ffseed = Fd * Fk * np.logical_not(Aseed)
u, v = np.where(np.triu(np.ones((nnode, nnode)), 1)) # get all row column combinations
A = copy.deepcopy(Aseed) # set the new variable
K = copy.deepcopy(Kseed) # set the new variable
Ff = copy.deepcopy(Ffseed) # set the new variable
statement = "From %g seed connections, adding %g connections" % (mseed,m)
print(statement)
opt_keep = np.zeros((m-mseed,nopt)) # initialise
for ii in range(mseed,m):
    C = np.append(0, np.cumsum(Ff[u, v]))
    r = np.sum(np.random.random()*C[-1] >= C)
    uu = u[r]
    vv = v[r]
    A[uu, vv] = A[vv, uu] = 1
    updateuu, = np.where(np.inner(A, A[:, uu]))
    np.delete(updateuu, np.where(updateuu == uu))
    np.delete(updateuu, np.where(updateuu == vv))
    c1 = np.append(A[:, uu], A[uu, :])
    for i in range(len(updateuu)):
        j = updateuu[i]
        c2 = np.append(A[:, j], A[j, :])

        use = np.logical_or(c1, c2)
        use[uu] = use[uu+nnode] = use[j] = use[j+nnode] = 0
        ncon = np.sum(c1[use]) + np.sum(c2[use])
        if ncon == 0:
            K[uu, j] = K[j, uu] = epsilon
        else:
            K[uu, j] = K[j, uu] = (2 / ncon * np.sum(np.logical_and(c1[use],c2[use])) +epsilon)
    updatevv, = np.where(np.inner(A, A[:, vv]))
    np.delete(updatevv, np.where(updatevv == uu))
    np.delete(updatevv, np.where(updatevv == vv))
    c1 = np.append(A[:, vv], A[vv, :])
    
    for i in range(len(updatevv)):
        j = updatevv[i]
        c2 = np.append(A[:, j], A[j, :])

        use = np.logical_or(c1, c2)
        use[vv] = use[vv+nnode] = use[j] = use[j+nnode] = 0
        ncon = np.sum(c1[use]) + np.sum(c2[use])
        if ncon == 0:
            K[vv, j] = K[j, vv] = epsilon
        else:
            K[vv, j] = K[j, vv] = (2 / ncon * np.sum(np.logical_and(c1[use],c2[use])) +epsilon)

    Fffinal = Fd * Fk * np.logical_not(A)
    
    # add an optimisation step
    
    #com = scipy.linalg.expm(A) # compute communicability
    #sumcom = np.sum(com) # sum communicability    
    
    strength = np.sum(A,axis=1) # get strength
    neg_sqrt = np.power(strength,-0.5) # negative sqrt
    S = np.diag(neg_sqrt) # diagonalise
    X = S.dot(A).dot(S) # form power term
    X = np.nan_to_num(X,nan=0) # ensure no nan values
    com = scipy.linalg.expm(X) # normalised communicability
    sumcom = np.sum(com) # sum normalised communicability
    opt_keep[ii] = sumcom
    for j in range(nopt): # nopt number of optimisation steps
        grad = np.gradient(A,sumcom) # compute the gradient
        W = A + alpha*grad[1] # walk down the gradient
        W[W<0] = 0 # limit to optimise positive weights
print('Simulation complete')

In [ ]:
# plot the outcome
plt.figure(figsize=(15,15))
subax1 = plt.subplot(321)
plt.imshow(Aseed)
subax2 = plt.subplot(322)
plt.imshow(Ffseed)
subax3 = plt.subplot(323)
plt.imshow(A)
subax4 = plt.subplot(324)
plt.imshow(Fffinal)
subax5 = plt.subplot(325)
plt.imshow(W)
subax6 = plt.subplot(326)
plt.hist(W[W>0])
plt.xlabel('Weight, w')
plt.ylabel('Frequency')
plt.title('Weight distribution')
subax6.spines['top'].set_visible(False)
subax6.spines['right'].set_visible(False)
# display the number of edges in target and final network as a sanity check
m,np.size(np.where(A.flat))//2 # number of connections